## ET Core Data Amalgamation

In [1]:
## Libraries

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import date
import datetime as dt

In [1]:
## Inputs - ET core data

core=pd.read_csv("01 Prime data\\etprimeusers-activity-core.csv",encoding = 'unicode_escape',error_bad_lines=False,engine='python',header=None)

NameError: name 'pd' is not defined

In [3]:
## Input - ET Prime data

prime_data=pd.read_csv("subdata_email.csv")
prime_data=prime_data[(prime_data["plan_id"]==1)|(prime_data["plan_id"]==2)|(prime_data["plan_id"]==3)|(prime_data["plan_id"]==8)]

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (5,43,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
# Resolving an issue
core.columns=core.columns.astype(str)

In [44]:
## Naming Columns 
core.columns.values[0]="datetime"
core.columns.values[1]="UUID"
core.columns.values[2]="user_email"
core.columns.values[3]="source"
core.columns.values[4]="MSID"
core.columns.values[5]="pagename"
core.columns.values[6]="section1"
core.columns.values[7]="section2"
core.columns.values[8]="section3"
core.columns.values[9]="section4"
core.columns.values[10]="title"
core.head()

,datetime,UUID,user_email,source,MSID,pagename,section1,section2,section3,section4,title
0,2019-12-05 09:31:51,1000105395825777,sumitbirdi@gmail.com,1,72376903,homepage,Home,Quick Read,NaN,NaN,ETNews
1,2019-12-06 12:29:39,1000105395825777,sumitbirdi@gmail.com,1,72393073,articleshow,Home,Top News,NaN,NaN,ETNews
2,2019-12-24 08:02:04,1000105395825777,sumitbirdi@gmail.com,1,0,homepage,Home,Top News,NaN,NaN,ETNews
3,2019-12-05 09:31:51,1000105395825777,sumitbirdi@gmail.com,1,0,homepage,Home,Top News,NaN,NaN,ETNews
4,2019-12-24 09:32:31,1000105395825777,sumitbirdi@gmail.com,1,0,homepage,Home,Top News,NaN,NaN,ETNews


In [46]:
core["pagename"]=core["pagename"].str.lower()

In [ ]:
# 32,543 user emails
# 32,570 UUID
# 3 Source -  1 - Mobile App; 0 - Web; 2 - WAP
# 120,272 MSID
# title - economictimes.indiatimes.com/ m.economictimes.com/ETNews

In [ ]:
# Articles Read
# Categories read - Article count (within dateframe)
# Loyalty - Definition? | 

In [7]:
# Ensuring date format consistency
core["datetime"]=pd.to_datetime(core["datetime"])

In [8]:
# Getting all values for one email in single row - deduping on email
prime_data.sort_values(by=["user_email","created_on"],ascending=[True,False],inplace=True)
prime_data.drop_duplicates("user_email",inplace=True)
prime_email=prime_data[["user_email","Renewal_Status","auto_renew","Status","plan_id","created_on","plan_expiration_details.0.subscriptionEndDate","plan_expiration_details.0.subscriptionTrialEndDate"]]                       

In [9]:
# Renaming prime oclumns for ease
prime_email.rename(columns={"plan_expiration_details.0.subscriptionEndDate":"sub_end_date","plan_expiration_details.0.subscriptionTrialEndDate":"trial_end_date" },inplace=True)

C:\Users\akhil.nair1\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [10]:
# Merging Prime start, end, trial and other status info with core data
core_prime=core.merge(prime_email,on="user_email",how="left")

In [11]:
# Format adjustment

#core_prime["created_on"]=pd.to_datetime(core_prime["created_on"])
core_prime["created_on"]=pd.to_datetime(core_prime["created_on"]).dt.tz_convert(None)
core_prime["sub_end_date"]=pd.to_datetime(core_prime["sub_end_date"])
core_prime["trial_end_date"]=pd.to_datetime(core_prime["trial_end_date"])

In [ ]:
# Users haven't read Prime
core_prime["plan_id"].unique()

In [12]:
# Correcting 2020 Dec dates to 2019 Dec dates

core_prime.loc[core_prime["datetime"]>'2020-02-05 19:40:06.000000000',"datetime"]="2019"+core_prime["datetime"].astype(str).str.slice(4,29)
core_prime["datetime"]=pd.to_datetime(core_prime["datetime"])

In [13]:
## Getting Months and date

core_prime["visit_date"]=core_prime["datetime"].astype(str).str.slice(0,10)
core_prime["visit_month"]=core_prime["datetime"].astype(str).str.slice(0,7)

In [47]:
## Total Visits

visits=pd.DataFrame(core_prime.groupby("user_email")["visit_date"].nunique())
visits=visits.reset_index()
visits.columns.values[0]="user_email"
visits.columns.values[1]="total_visits"

In [49]:
## Total Articles

articles_valid=core_prime[core_prime["MSID"]>0]

articles=pd.DataFrame(articles_valid.groupby("user_email")["MSID"].nunique())
articles=articles.reset_index()
articles.columns.values[0]="user_email"
articles.columns.values[1]="total_articles"

In [59]:
articles.sort_values(by=["total_articles"],ascending=False)

,user_email,total_articles
24631,satishkumar.bhi@gmail.com,8082
18479,pareshbagwe5@gmail.com,7117
23955,sandeepbgupta@yahoo.com,6950
21096,rajeshbshah1@gmail.com,6656
3545,anshulrana00551@gmail.com,6534
25094,sgpatel11@yahoo.co.in,6494
13247,kishoreshivani@yahoo.co.in,6344
10146,guptarishabh88@gmail.com,6269
25900,shoppersin@gmail.com,6078
11068,imaxoverseas@gmail.com,6064


In [50]:
# Count of unique sections - by user

x= ["markets","news","industry","small biz","wealth","tech","magazines","mf","daily brief","nse","portfolio","jobs","morning brief","deeplink","nri","hindi","topic","evening brief","watchlist","indices","opinion"]
a=list(x)

cat_valid=core_prime[core_prime["section1"].isin(a)]

cat=pd.DataFrame(cat_valid.groupby("user_email")["section1"].nunique())
cat=cat.reset_index()
cat.columns.values[0]="user_email"
cat.columns.values[1]="section_count"

In [51]:
## creating datasets for first 7, last 7 and within trial

core_prime["+7_days"]=core_prime["created_on"]+pd.DateOffset(days=7)
core_prime["-7_days"]=core_prime["sub_end_date"]-pd.DateOffset(days=7)

first_7=core_prime[(core_prime["datetime"]>=core_prime["created_on"])&(core_prime["datetime"]<=core_prime["+7_days"])]
last_7=core_prime[(core_prime["datetime"]<=core_prime["sub_end_date"])&(core_prime["datetime"]>=core_prime["-7_days"])]

In [52]:
## ----------first_7------------------

## Total Visits

visits_f7=pd.DataFrame(first_7.groupby("user_email")["visit_date"].nunique())
visits_f7=visits_f7.reset_index()
visits_f7.columns.values[0]="user_email"
visits_f7.columns.values[1]="f7_visits"


## Total Articles

articles_valid_f7=first_7[first_7["MSID"]>0]

articles_f7=pd.DataFrame(articles_valid_f7.groupby("user_email")["MSID"].nunique())
articles_f7=articles_f7.reset_index()
articles_f7.columns.values[0]="user_email"
articles_f7.columns.values[1]="f7_articles"


# Count of unique sections - by user

cat_valid_f7=first_7[first_7["section1"].isin(a)]

cat_f7=pd.DataFrame(cat_valid_f7.groupby("user_email")["section1"].nunique())
cat_f7=cat_f7.reset_index()
cat_f7.columns.values[0]="user_email"
cat_f7.columns.values[1]="f7_section_count"

In [53]:
## ----------last_7------------------

## Total Visits

visits_l7=pd.DataFrame(last_7.groupby("user_email")["visit_date"].nunique())
visits_l7=visits_l7.reset_index()
visits_l7.columns.values[0]="user_email"
visits_l7.columns.values[1]="l7_visits"


## Total Articles

articles_valid_l7=last_7[last_7["MSID"]>0]

articles_l7=pd.DataFrame(articles_valid_l7.groupby("user_email")["MSID"].nunique())
articles_l7=articles_l7.reset_index()
articles_l7.columns.values[0]="user_email"
articles_l7.columns.values[1]="l7_articles"


# Count of unique sections - by user

cat_valid_l7=last_7[last_7["section1"].isin(a)]

cat_l7=pd.DataFrame(cat_valid_l7.groupby("user_email")["section1"].nunique())
cat_l7=cat_l7.reset_index()
cat_l7.columns.values[0]="user_email"
cat_l7.columns.values[1]="l7_section_count"

In [54]:
## Merging the visits, articles and sections visited count

core_agg=visits.merge(articles,on="user_email",how="left").merge(cat,on="user_email",how="left").merge(visits_l7,on="user_email",how="left").merge(articles_l7,on="user_email",how="left").merge(cat_l7,on="user_email",how="left").merge(visits_f7,on="user_email",how="left").merge(articles_f7,on="user_email",how="left").merge(cat_f7,on="user_email",how="left")

In [55]:
## Getting Renewal Statuses from Prime data

prime_data.sort_values("user_email",inplace=True)
prime_data.drop_duplicates("user_email",inplace=True)

prime_small=prime_data[["user_email","Status","Renewal_Status"]]
core_pre_final=core_agg.merge(prime_small,on="user_email",how="left")

In [56]:
## Getting number of days between the first and last visit

s_max=core_prime.sort_values(by=["user_email","datetime"],ascending=[True,False])
s_max.drop_duplicates(subset=["user_email"],inplace=True)

s_min=core_prime.sort_values(by=["user_email","datetime"],ascending=[True,True])
s_min.drop_duplicates(subset=["user_email"],inplace=True)

s_max=s_max[["user_email","datetime"]]
s_max.rename(columns={"datetime":"last_visit"},inplace=True)

s_min=s_min[["user_email","datetime"]]
s_min.rename(columns={"datetime":"first_visit"},inplace=True)

s_merge=s_max.merge(s_min,on="user_email",how="left")
s_merge["days_visited"]=s_merge["last_visit"]-s_merge["first_visit"]
s_merge["days_visited"]=s_merge["days_visited"].dt.days+1

In [57]:
## Merging all and exporting

core_final=core_pre_final.merge(s_merge,on="user_email",how="left")
core_final.to_csv("01 core agg data.csv",index=False)